In [ ]:
# Load the auto reload extension to automatically reload modules when files on disk are updated
%load_ext autoreload
# it will automatically be reloaded without the need to restart the kernel.
%autoreload 

In [7]:
import sys
sys.path.append("../agrilearn/submodules/commons/")
sys.path.append("../agrilearn/submodules/crop_rnn/")
sys.path.append("../agrilearn/")

import boto3
import geopandas as gpd
import os
from io import BytesIO
import pandas as pd
from folium import GeoJson
import folium
import numpy as np
from tqdm import tqdm
from glob import glob
#import local modules
import sys
from eolearn.core import EOPatch

#from agrilearn.commons.s3 import s3_utils
#from agrilearn.crop_classification import evalutate_utils
#from agrilearn.crop_classification import yaml_utils
#from agrilearn.crop_classification.processing import check_input_model_eopatches_features_based_geopackage, check_processed_eopatches_features_based_geopackage

In [ ]:
config = yaml_utils.load_config("/agrilearn_app/output/config/experiment_12.yaml")

In [ ]:
index=0
DATASET_NAME = config['evaluate'][index]['dataset_name']
GEOPACKAGE_PATH = config['evaluate'][index]['geopackage_test_data_path']
EOPATCH_PATH = config['evaluate'][index]['eopatch_folder']
LABEL_MONITORING_CLASS = config['evaluate'][index]['label_true']
LABEL_EOPATCH_LOCATION = config['evaluate'][index]['label_eopatch_location']

In [ ]:
DATASET_NAME

In [ ]:
EOPATCH_PATH

## 1. Read Geopackage

In [ ]:
GEOPACKAGE_PATH="../../nicksson-datasets/base/"

In [ ]:
#CORN_73080_SOYBEAN_29670_COTTON_1632_RICE_1172.gpkg
# GEOPACKAGE_PATH = "/agrilearn_app/datasets/experiment_12/geopackage/SOYBEAN_48257_CORN_33809_SUGAR_CANE_17640_WHEAT_4226_COTTON_3178_RICE_925_test.gpkg"
# EOPATCH_PATH = "/agrilearn_app/datasets/experiment_12/eopatch/input_model"


# GEOPACKAGE_PATH = "/agrilearn_app/datasets/soja_2022_2023/geopackage/raw/soja_mar_66835.gpkg"
# EOPATCH_PATH =  "/agrilearn_app/datasets/soja_2022_2023/eopatch/processed/"

# LABEL_MONITORING_CLASS = 'monitoring_class'
# LABEL_EOPATCH_LOCATION = "eopath_location" # 'sentinel_eopatch_current' #eopath_location

In [ ]:
df = gpd.read_file(GEOPACKAGE_PATH)

In [ ]:
df[LABEL_MONITORING_CLASS].value_counts()

## 2. Check Class in eopatch path

In [ ]:
df_results, df_overview = check_processed_eopatches_features_based_geopackage(df, 
                                                          eopatch_path=EOPATCH_PATH,
                                                          label_eopatch_path=LABEL_EOPATCH_LOCATION)

In [ ]:
full_df_processed = pd.merge(df_results, 
                   df, 
                   left_on='geopackage', 
                   right_on=LABEL_EOPATCH_LOCATION)

In [ ]:
full_df_processed['eopatch_exists'].value_counts()

In [ ]:
full_df_processed[full_df_processed['eopatch_exists']==True].shape

In [ ]:
full_df_processed[full_df_processed['eopatch_exists']==False].shape

In [ ]:
full_df_processed.to_csv(os.path.join(config['data']['experiment_path'], f'processed_eopatch_{DATASET_NAME}.csv'))
df_overview.to_csv(os.path.join(config['data']['experiment_path'], f'processed_eopatch_overview_{DATASET_NAME}.csv'))

In [ ]:
EOPATCH_PATH =  "/agrilearn_app/datasets/soja_2022_2023/eopatch/input_model/"

In [ ]:
#check_input_model_eopatches_features_based_geopackage
df_results, df_overview = check_input_model_eopatches_features_based_geopackage(df, 
                                                          eopatch_path=EOPATCH_PATH,
                                                          label_eopatch_path=LABEL_EOPATCH_LOCATION)

In [ ]:
full_df_input_model = pd.merge(df_results, 
                   df, 
                   left_on='geopackage', 
                   right_on=LABEL_EOPATCH_LOCATION)

In [ ]:
full_df_input_model['eopatch_exists'].value_counts()

In [ ]:
full_df_input_model[full_df_input_model['eopatch_exists']==True].shape

In [ ]:
full_df_input_model[full_df_input_model['eopatch_exists']==False].shape

In [ ]:
full_df_input_model[full_df_input_model['eopatch_exists']==False].shape

In [ ]:
full_df_input_model.to_csv(os.path.join(config['data']['experiment_path'], f'input_model_{DATASET_NAME}.csv'))
df_overview.to_csv(os.path.join(config['data']['experiment_path'], f'input_model_overview_{DATASET_NAME}.csv'))

In [ ]:
# def update_eopatch_labels_based_string_eopatch_dir(base_path):
#     dic_erros = {}
#     list_monitoring_class_before = {}
#     list_monitoring_class_after = {}

#     # Iterar sobre cada subdiretório no diretório base_path
#     for root, dirs, files in os.walk(base_path):
#         for dir_name in tqdm(dirs, desc="Processando diretórios"):
#             print(dir_name)

#     #         if "monitoring_class_" in file and "_epsg4326" in file:
#     #             try:
#     #                 # Extrair a string entre monitoring_class_ e _epsg4326
#     #                 match = re.search(r'monitoring_class_(.*?)_epsg4326', file)
#     #                 if match:
#     #                     monitoring_class = match.group(1)

#     #                     # Construir o caminho completo do eopatch
#     #                     final_eopatch_path = os.path.join(root, file)
#     #                     eopatch = EOPatch.load(final_eopatch_path)

#     #                     # Salvar a classe de monitoramento antes da substituição
#     #                     lista = np.array(eopatch['meta_info']['LABELS'])
#     #                     list_monitoring_class_before[final_eopatch_path] = np.unique(lista)[0]

#     #                     # Atribuir a string extraída ao dado do eopatch
#     #                     eopatch['meta_info']['LABELS'] = [monitoring_class]

#     #                     # Salvar o EOPatch no disco
#     #                     eopatch.save(final_eopatch_path, overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)

#     #                     # Salvar a classe de monitoramento após a substituição
#     #                     lista = np.array(eopatch['meta_info']['LABELS'])
#     #                     list_monitoring_class_after[final_eopatch_path] = np.unique(lista)[0]

#     #                     print(f"EOPatch salvo em: {final_eopatch_path}")

#     #                 else:
#     #                     print(f"Formato de arquivo inesperado: {file}")

#     #             except Exception as e:
#     #                 dic_erros[final_eopatch_path] = e

#     # # Criar DataFrames para os resultados
#     # df_erros = pd.DataFrame(list(dic_erros.items()), columns=['eopatch_location', 'error'])
#     # df_eopatch_before = pd.DataFrame(list(list_monitoring_class_before.items()), columns=['eopatch_location', 'monitoring_class_before'])
#     # df_eopatch_after = pd.DataFrame(list(list_monitoring_class_after.items()), columns=['eopatch_location', 'monitoring_class_after'])

#     # return df_erros, df_eopatch_before, df_eopatch_after


# # df_erros, df_eopatch_before, df_eopatch_after = update_eopatch_labels(base_path)

In [ ]:
# base_path = "/agrilearn_app/datasets/experiment_12/eopatch/input_model/"
# update_eopatch_labels_based_string_eopatch_dir(base_path)

In [ ]:
# df_eopatch, df_erros = check_eopaches_based_geopackage(df.iloc[:100], eopatch_path=EOPATCH_PATH)

In [ ]:
# if len(df_eopatch.columns) == 4:
#     df_eopatch.columns = ['eopatch_location', 'dataset_source', 'monitoring_class_eopatch', 'monitoring_class_geopackage']

# if len(df_erros.columns) == 4:
#     df_erros.columns = ['eopatch_location', 'dataset_source', 'error', 'monitoring_class_geopackage']

In [ ]:

# df_eopatch['culture_based_path']

In [ ]:
# dic_erros = {}
# list_monitoring_class_before = {}
# list_monitoring_class_after = {}

# # Dicionário de substituição
# dic_replace = {
#     'soybean': 'SOYBEAN', 
#     'soja': 'SOYBEAN',
#     'corn': 'CORN',
#     'milho': 'CORN',
#     'cotton': 'COTTON',
#     'algodao': 'COTTON',
#     'sugar_cane': 'SUGAR_CANE',
#     'cana': 'SUGAR_CANE',
#     'rice': 'RICE',
#     'arroz': 'RICE'
# }

# # Iterar sobre cada linha do DataFrame
# for index, row in tqdm(df.iterrows(), total=df.shape[0]):
#     try:
#         eopatch_location_id = row[label_eopatch_path]
#         final_eopatch_path = os.path.join(EOPATCH_PATH,
#                                           eopatch_location_id,
#                                           "eopatch_0_col-0_row-0")

#         eopatch = EOPatch.load(final_eopatch_path)

#         lista = np.array(eopatch['meta_info']['LABELS'])
#         list_monitoring_class_before[eopatch_location_id] = np.unique(lista)[0]

#         # # Substituir os valores conforme o dicionário dic_replace
#         # matriz = np.array(eopatch['meta_info']['LABELS'])
#         # for key, value in dic_replace.items():
#         #     matriz = np.where(matriz == key, value, matriz)
#         # eopatch['meta_info']['LABELS'] = matriz.tolist()

#         # # Salvar o EOPatch no disco
#         # final_eopatch_path = final_eopatch_path.replace('input_model-v2', 'input_model')
#         # eopatch.save(final_eopatch_path, overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)

#         # lista = np.array(eopatch['meta_info']['LABELS'])
#         # list_monitoring_class_after[eopatch_location_id] = np.unique(lista)[0]
        
#         #print(f"EOPatch salvo em: {final_eopatch_path}")

#     except Exception as e:
#         dic_erros[final_eopatch_path] = e

# df_erros = pd.DataFrame(list(dic_erros.items()), columns=['Chave', 'Valor'])
# df_eopatch_before = pd.DataFrame(list(list_monitoring_class_before.items()), columns=['eopatch_location_id', 'monitoring_class'])
# df_eopatch_after = pd.DataFrame(list(list_monitoring_class_after.items()), columns=['eopatch_location_id', 'monitoring_class'])

In [ ]:
# dic_erros = {}
# list_monitoring_class = {}

# eopatch_locations = df['eopath_location'].unique()
# len(eopatch_locations)

# for eopatch_location_id in tqdm(eopatch_locations):
#     try:

#         final_eopatch_path = os.path.join(EOPATCH_PATH,
#                                           eopatch_location_id,
#                                           "eopatch_0_col-0_row-0")

#         eopatch = EOPatch.load(final_eopatch_path)

#         lista = np.array(eopatch['meta_info']['LABELS'])
#         list_monitoring_class[eopatch_location_id] = np.unique(lista)[0]


#         # # #Replace Eopatch class name and Save
#         # matriz = np.array(eopatch['meta_info']['LABELS'])
#         # nova_list = np.where(matriz == "algodao", "COTTON", matriz).tolist()
#         # eopatch['meta_info']['LABELS'] = nova_list


#         # # # Salve o EOPatch no disco
#         # final_eopatch_path = final_eopatch_path.replace('input_model-v2', 'input_model')
#         # eopatch.save(final_eopatch_path, overwrite_permission=OverwritePermission.OVERWRITE_PATCH)
#         # # print(f"EOPatch salvo em: {final_eopatch_path}")
        
#     except Exception as e:
#         dic_erros[final_eopatch_path] = e

# df_erros = pd.DataFrame(list(dic_erros.items()), columns=['Chave', 'Valor'])
# df_eopatch = pd.DataFrame(list(list_monitoring_class.items()), columns=['eopatch_location_id', 'monitoring_class'])

In [ ]:
final_eopatch_path

In [ ]:
#df_erros.to_csv('eopatch_erros_soja_mar_66835.csv')

In [ ]:
# replacement_dict = {
#     'SUGAR_CANE': 'cana',
# }

# df['monitoring_class'] = df['monitoring_class'].replace(replacement_dict)

In [ ]:
# eopatch_utils.create_local_eopatch_path(df, 
#                           path_local="/agrilearn_app/datasets/eopatchs/modeling/experiment_01/",
#                           label_monitoring_class='monitoring_class',
#                           label_dataset_part='dataset_part',
#                           label_eopatch_location='eopath_location',
#                           )

In [ ]:
# path_id = {}
# failed = {}

# for index, row in tqdm(df.iterrows(), total=len(df)):

#     try:
    
#         eopatch = EOPatch.load(row['local_eopatch_path'], 
#                                lazy_loading=True)
    
#         path_id[row['local_eopatch_path']] = eopatch.mask['SCL'].shape[0]
#     except Exception as e:
#         failed[row['local_eopatch_path']] = e
    
# df_checked = pd.DataFrame(list(path_id.items()), columns=['path', "shape"])
# df_failed = pd.DataFrame(list(failed.items()), columns=['path', "shape"])